In [1]:
from tensorflow import keras

## Transfer Learning 을 통한 가위-바위-보 분류 데이터 셋 분류 성능 개선

### Step 1. Input tensor 와 Target tensor 준비(훈련데이터)

(1) 가위-바위-보 데이터셋 다운로드

In [2]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip'

path_to_zip = keras.utils.get_file('rps.zip',
                                   origin=url,
                                   extract=True,
                                   cache_dir='/content')

200695808/200682221 [==============================] - 3s 0us/step


In [3]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip'

path_to_zip = keras.utils.get_file('rps_test.zip',
                                   origin=url,
                                   extract=True,
                                   cache_dir='/content')

29532160/29516758 [==============================] - 0s 0us/step


(2) ImageDataGenerator를 이용해 이미지 파일을 load 하기 위한 경로 지정

In [4]:
train_dir = '/content/datasets/rps'
test_dir = '/content/datasets/rps-test-set'

(3) ImageDataGenerator 객체 생성  
* 객체 생성 시 rescale 인자를 이용하여 텐서 내 원소의 범위를 [0 ~ 255] => [0 ~ 1] 로 ReScaling 진행

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# 모든 이미지를 1/255로 스케일을 조정합니다
train_datagen = ImageDataGenerator(rescale=1./255,
                                   validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

* .flow_from_directory() 메서드를 이용하여 학습데이터와 검증데이터를 위한 DirectoryIterator 객체 생성

In [7]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=20,
        shuffle=True,
        class_mode='categorical',
        subset='training',
        seed=7)

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=20,
        shuffle=True,
        class_mode='categorical',
        subset='validation',
        seed=7)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical')

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


### Step 2. VGG16을 Backbone 으로 하는 모델 디자인 및 학습 정보 설정

(1) Pre-trained 된 VGG16 모델 객체 생성
  * imagenet 데이터를 이용해 학습된 모델 객체 생성
  * classification layer 제외

In [8]:
from tensorflow.keras.applications import VGG16

In [9]:
conve_base = VGG16(include_top=False,
                   weights='imagenet',
                   input_shape=(224, 224, 3))

58900480/58889256 [==============================] - 1s 0us/step


In [10]:
conve_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

(2) VGG16 Backbone 모델에 classification layer 추가

In [11]:
model = keras.Sequential()

In [12]:
model.add(conve_base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6422784   
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 21,138,243
Trainable params: 21,138,243
Non-trainable params: 0
_________________________________________________________________


(3) VGG16 Backbone 모델의 가중치 동결(학습대상 가중치에서 제외)

* VGG16 Backbone 모델의 가중치 동결 및 동결 후 학습대상 파라미터 개수 출력

In [14]:
len(model.trainable_weights)

30

In [15]:
conve_base.trainable = False

In [16]:
len(model.trainable_weights)

4

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6422784   
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 21,138,243
Trainable params: 6,423,555
Non-trainable params: 14,714,688
_________________________________________________________________


(4) 학습을 위한 설정 정보 지정

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(learning_rate=2e-5),
              metrics=['accuracy'])

### Step 3. 모델에 데이터 generator 연결 후 학습 
  * model.fit() 이용하여 데이터 연결 및 학습시키기
  * 학습 과정은 history 변수에 저장

In [19]:
history = model.fit(
      train_generator,
      steps_per_epoch=len(train_generator),
      epochs=30,
      validation_data=validation_generator,
      validation_steps=len(validation_generator))

Epoch 1/30
101/101 [==============================] - 68s 336ms/step - loss: 0.2082 - accuracy: 0.9648 - val_loss: 0.0431 - val_accuracy: 1.0000
Epoch 2/30
101/101 [==============================] - 27s 266ms/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.0077 - val_accuracy: 1.0000
Epoch 3/30
101/101 [==============================] - 27s 268ms/step - loss: 9.3590e-04 - accuracy: 1.0000 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 4/30
101/101 [==============================] - 27s 269ms/step - loss: 1.5485e-04 - accuracy: 1.0000 - val_loss: 9.9714e-04 - val_accuracy: 1.0000
Epoch 5/30
101/101 [==============================] - 27s 269ms/step - loss: 3.6376e-05 - accuracy: 1.0000 - val_loss: 7.3093e-04 - val_accuracy: 1.0000
Epoch 6/30
101/101 [==============================] - 27s 269ms/step - loss: 9.2986e-06 - accuracy: 1.0000 - val_loss: 3.4259e-04 - val_accuracy: 1.0000
Epoch 7/30
101/101 [==============================] - 27s 269ms/step - loss: 2.5158e-06 - accuracy: 1.

### Step 4. 테스트 데이터 셋을 통한 모델의 성능 평가

In [20]:
loss, accuracy = model.evaluate(test_generator)

19/19 [==============================] - 8s 437ms/step - loss: 0.1616 - accuracy: 0.9462


In [21]:
del model